In [17]:
API_KEY = "aac405e628f9c30a047d3de13192a7f7"

In [4]:
import pandas as pd
df = pd.read_csv('Desktop/india_monitoring_locations_filtered.csv')

In [5]:
df

,id,name,city,country,latitude,longitude,parameters
0,12,SPARTAN - IIT Kanpur,SPARTAN - IIT Kanpur,IN,26.519000,80.233000,NaN
1,13,"Delhi Technological University, Delhi - CPCB","Delhi Technological University, Delhi - CPCB",IN,28.744000,77.120000,NaN
2,15,IGI Airport,IGI Airport,IN,28.560000,77.094000,NaN
3,16,Civil Lines,Civil Lines,IN,28.678700,77.226200,NaN
4,17,"R K Puram, Delhi - DPCC","R K Puram, Delhi - DPCC",IN,28.563262,77.186937,NaN
...,...,...,...,...,...,...,...
657,3409526,"Panchawati_Pashan, Pune - IITM","Panchawati_Pashan, Pune - IITM",IN,18.536457,73.805454,NaN
658,3409527,"SIPCOT Industrial Park, Perundurai - TNPCB","SIPCOT Industrial Park, Perundurai - TNPCB",IN,11.258242,77.552761,NaN
659,3409528,"Savta Mali Nagar, Pimpri-Chinchwad - IITM","Savta Mali Nagar, Pimpri-Chinchwad - IITM",IN,18.614767,73.799516,NaN
660,3409529,"Mahishkapur Road_B-Zone, Durgapur - WBPCB","Mahishkapur Road_B-Zone, Durgapur - WBPCB",IN,23.567923,87.306843,NaN


In [6]:
df_guwahati = df[df['city'].str.contains('Guwahati', case=False, na=False)]
print(df_guwahati)


          id                             name  \
171     6941  Railway Colony, Guwahati - APCB   
293    10903  Railway Colony, Guwahati - APCB   
339    42240      Pan Bazaar, Guwahati - APCB   
452   361411            IITG, Guwahati - PCBA   
495  3409360            IITG, Guwahati - PCBA   
525  3409390    LGBI Airport, Guwahati - PCBA   

                                city country   latitude  longitude  parameters  
171  Railway Colony, Guwahati - APCB      IN  26.144500  91.736200         NaN  
293  Railway Colony, Guwahati - APCB      IN  26.181742  91.780630         NaN  
339      Pan Bazaar, Guwahati - APCB      IN  26.187500  91.744194         NaN  
452            IITG, Guwahati - PCBA      IN  26.202864  91.700464         NaN  
495            IITG, Guwahati - PCBA      IN  26.202864  91.700464         NaN  
525    LGBI Airport, Guwahati - PCBA      IN  26.108870  91.589544         NaN  


In [9]:
stations = df_guwahati.to_dict(orient='records')

# Now df_list is a list of dictionaries, 
# where each dictionary corresponds to a row of the DataFrame.
for item in df_list:
    print(item)



{'id': 6941, 'name': 'Railway Colony, Guwahati - APCB', 'city': 'Railway Colony, Guwahati - APCB', 'country': 'IN', 'latitude': 26.1445, 'longitude': 91.7362, 'parameters': nan}
{'id': 10903, 'name': 'Railway Colony, Guwahati - APCB', 'city': 'Railway Colony, Guwahati - APCB', 'country': 'IN', 'latitude': 26.181742, 'longitude': 91.78063, 'parameters': nan}
{'id': 42240, 'name': 'Pan Bazaar, Guwahati - APCB', 'city': 'Pan Bazaar, Guwahati - APCB', 'country': 'IN', 'latitude': 26.1875, 'longitude': 91.744194, 'parameters': nan}
{'id': 361411, 'name': 'IITG, Guwahati - PCBA', 'city': 'IITG, Guwahati - PCBA', 'country': 'IN', 'latitude': 26.2028636, 'longitude': 91.70046436, 'parameters': nan}
{'id': 3409360, 'name': 'IITG, Guwahati - PCBA', 'city': 'IITG, Guwahati - PCBA', 'country': 'IN', 'latitude': 26.2028636, 'longitude': 91.70046436, 'parameters': nan}
{'id': 3409390, 'name': 'LGBI Airport, Guwahati - PCBA', 'city': 'LGBI Airport, Guwahati - PCBA', 'country': 'IN', 'latitude': 26.10

In [39]:
import requests
import datetime
import time
import pandas as pd

BASE_URL = "http://api.openweathermap.org/data/2.5/air_pollution/history"

end_date = datetime.date.today()
start_date = end_date - datetime.timedelta(days=90)

def to_timestamp(dt):
    """Convert a datetime object to a Unix timestamp."""
    return int(dt.timestamp())

def fetch_daily_air_quality(station, day):
    """
    Fetch the historical air quality data for a specific station on a given day.
    Retrieves the AQI as well as hourly pollutant values for:
      - co, no, no2, o3, so2, pm2_5, pm10, nh3
    and computes daily averages.
    """
    day_start = datetime.datetime.combine(day, datetime.time.min)
    day_end = datetime.datetime.combine(day, datetime.time.max)
    
    params = {
        "lat": station["latitude"],
        "lon": station["longitude"],
        "start": to_timestamp(day_start),
        "end": to_timestamp(day_end),
        "appid": API_KEY
    }
    
    response = requests.get(BASE_URL, params=params)
    if response.status_code == 200:
        data = response.json()
        
        # Lists to hold hourly data
        hourly_aqi = []
        # Define the pollutant components to extract
        components_keys = ["co", "no", "no2", "o3", "so2", "pm2_5", "pm10", "nh3"]
        hourly_components = {key: [] for key in components_keys}
        
        for entry in data.get("list", []):
            # Extract AQI value from "main"
            if "main" in entry and "aqi" in entry["main"]:
                hourly_aqi.append(entry["main"]["aqi"])
            
            # Extract pollutant components if available
            if "components" in entry:
                for key in components_keys:
                    if key in entry["components"]:
                        hourly_components[key].append(entry["components"][key])
        
        # Compute daily averages for AQI and each pollutant
        daily_avg_aqi = sum(hourly_aqi) / len(hourly_aqi) if hourly_aqi else None
        daily_avg_components = {}
        for key, values in hourly_components.items():
            daily_avg_components[key] = sum(values) / len(values) if values else None
        
        return daily_avg_aqi, daily_avg_components
    else:
        print(f"Error on {day} for {station['name']}: {response.status_code} - {response.text}")
        return None, {}

def collect_data(stations):
    """
    Iterates over stations and days, fetching the daily average AQI and pollutant values,
    and returns a list of dictionaries (one per day per station).
    """
    
    records = []
    
    for station in stations:
        print(f"\nFetching air quality data for {station['name']} (lat: {station['latitude']}, lon: {station['longitude']})")
        current_date = start_date
        while current_date <= end_date:
            avg_aqi, avg_components = fetch_daily_air_quality(station, current_date)
            
            # Build a record (dictionary) with the station name, date, and pollutant averages
            record = {
                "station": station["name"],
                "date": current_date,
                "avg_aqi": avg_aqi
            }
            # Add each pollutant component to the record
            for comp, value in avg_components.items():
                record[comp] = value
            
            records.append(record)
            
            print(f"  {current_date}: AQI={avg_aqi}, Pollutants={avg_components}")
            
            # Advance to the next day
            current_date += datetime.timedelta(days=1)
            # Pause briefly to respect API rate limits (adjust as necessary)
            time.sleep(1)
    
    return records

# Collect the data and convert it into a DataFrame
data_records = collect_data(stations)
df = pd.DataFrame(data_records)
print("\nDataFrame with daily average pollutant values:")
print(df)



Fetching air quality data for Railway Colony, Guwahati - APCB (lat: 26.1445, lon: 91.7362)
  2024-12-20: AQI=4.291666666666667, Pollutants={'co': 1093.8445833333333, 'no': 3.0749999999999997, 'no2': 32.312083333333334, 'o3': 29.38, 'so2': 10.292916666666667, 'pm2_5': 76.72291666666666, 'pm10': 94.26875, 'nh3': 21.15875}
  2024-12-21: AQI=4.041666666666667, Pollutants={'co': 893.85125, 'no': 1.2325, 'no2': 24.289583333333336, 'o3': 40.117916666666666, 'so2': 6.937083333333334, 'pm2_5': 62.20416666666667, 'pm10': 76.50875, 'nh3': 16.8775}
  2024-12-22: AQI=3.5833333333333335, Pollutants={'co': 714.1620833333333, 'no': 0.7708333333333334, 'no2': 15.82, 'o3': 43.61541666666667, 'so2': 4.193333333333333, 'pm2_5': 50.21125, 'pm10': 59.96666666666667, 'nh3': 8.43375}
  2024-12-23: AQI=3.75, Pollutants={'co': 751.5741666666667, 'no': 1.6341666666666665, 'no2': 17.791666666666668, 'o3': 36.90125, 'so2': 3.8341666666666665, 'pm2_5': 50.59791666666666, 'pm10': 62.85666666666666, 'nh3': 7.76625}


In [40]:
df.head()



,station,date,avg_aqi,co,no,no2,o3,so2,pm2_5,pm10,nh3
0,"Railway Colony, Guwahati - APCB",2024-12-20,4.291667,1093.844583,3.075000,32.312083,29.380000,10.292917,76.722917,94.268750,21.158750
1,"Railway Colony, Guwahati - APCB",2024-12-21,4.041667,893.851250,1.232500,24.289583,40.117917,6.937083,62.204167,76.508750,16.877500
2,"Railway Colony, Guwahati - APCB",2024-12-22,3.583333,714.162083,0.770833,15.820000,43.615417,4.193333,50.211250,59.966667,8.433750
3,"Railway Colony, Guwahati - APCB",2024-12-23,3.750000,751.574167,1.634167,17.791667,36.901250,3.834167,50.597917,62.856667,7.766250
4,"Railway Colony, Guwahati - APCB",2024-12-24,3.666667,867.287917,2.065417,23.366250,33.017500,6.675833,53.242917,68.955833,16.334167


In [41]:
df

,station,date,avg_aqi,co,no,no2,o3,so2,pm2_5,pm10,nh3
0,"Railway Colony, Guwahati - APCB",2024-12-20,4.291667,1093.844583,3.075000,32.312083,29.380000,10.292917,76.722917,94.268750,21.158750
1,"Railway Colony, Guwahati - APCB",2024-12-21,4.041667,893.851250,1.232500,24.289583,40.117917,6.937083,62.204167,76.508750,16.877500
2,"Railway Colony, Guwahati - APCB",2024-12-22,3.583333,714.162083,0.770833,15.820000,43.615417,4.193333,50.211250,59.966667,8.433750
3,"Railway Colony, Guwahati - APCB",2024-12-23,3.750000,751.574167,1.634167,17.791667,36.901250,3.834167,50.597917,62.856667,7.766250
4,"Railway Colony, Guwahati - APCB",2024-12-24,3.666667,867.287917,2.065417,23.366250,33.017500,6.675833,53.242917,68.955833,16.334167
...,...,...,...,...,...,...,...,...,...,...,...
541,"LGBI Airport, Guwahati - PCBA",2025-03-16,2.750000,791.350833,0.127500,9.768750,68.790833,4.997917,29.778750,35.332917,7.310000
542,"LGBI Airport, Guwahati - PCBA",2025-03-17,3.875000,944.336250,0.586250,12.342500,92.186250,5.654167,60.948750,71.775833,12.542917
543,"LGBI Airport, Guwahati - PCBA",2025-03-18,3.708333,454.923750,0.043750,3.147917,137.745833,5.856667,39.878750,48.540000,8.215833
544,"LGBI Airport, Guwahati - PCBA",2025-03-19,4.375000,708.740000,0.116250,5.360000,119.537500,5.284167,71.873750,86.882917,21.480000


In [42]:
import numpy as np
import pandas as pd

def create_sequences(df, pollutants, window_size=7):
    """
    Build sliding windows of length 'window_size' for each station,
    returning X, y for multi-output regression.
    
    X will have shape: (num_samples, window_size * num_features)
    y will have shape: (num_samples, num_features)
    """
    X_list = []
    Y_list = []
    
    # Group by station so we don't mix sequences across stations
    for st, group in df.groupby("station"):
        # Sort by date to ensure chronological order
        group = group.sort_values("date").reset_index(drop=True)
        
        # Drop rows with NaN in any of the pollutant columns
        group = group.dropna(subset=pollutants)
        
        # If not enough rows for a 7-day window + next day
        if len(group) <= window_size:
            continue
        
        # Extract just the pollutant columns as a NumPy array
        # Shape: (num_days_for_station, num_pollutants)
        arr = group[pollutants].values
        
        # Slide over the array to create 7-day inputs -> next-day target
        for i in range(len(arr) - window_size):
            # 7-day window of shape (7, num_pollutants) -> flatten into 1D
            X_window = arr[i : i + window_size].flatten()
            # The day after the window is the target
            Y_target = arr[i + window_size]
            
            X_list.append(X_window)
            Y_list.append(Y_target)
    
    X = np.array(X_list)
    y = np.array(Y_list)
    return X, y


In [43]:
# Ensure sorted by station, then date
df = df.sort_values(["station", "date"]).reset_index(drop=True)

# Define the columns you want to predict (multi-output)
pollutants = ["co", "no", "no2", "o3", "so2", "pm2_5", "pm10", "nh3", "avg_aqi"]

# Build the sequences
X, y = create_sequences(df, pollutants=pollutants, window_size=7)
print("X shape:", X.shape)  # (num_samples, 7 * num_pollutants)
print("y shape:", y.shape)  # (num_samples, num_pollutants)


X shape: (512, 63)
y shape: (512, 9)


In [49]:
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler

# Train-test split (no shuffle if your data is strictly time-series; 
# but for demonstration, we often do shuffle=False for time-series)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Scale features (recommended for neural networks)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Create a lightweight MLP
# - hidden_layer_sizes=(16,) => single hidden layer with 16 neurons
# - If you want a single-layer linear perceptron (no hidden layers), use hidden_layer_sizes=()
model = MLPRegressor(
    hidden_layer_sizes=(16,),
    activation="relu",   # or "identity" if you want a purely linear output
    max_iter=5000,
    random_state=42
)

# Train the model
model.fit(X_train_scaled, y_train)

# Evaluate
train_score = model.score(X_train_scaled, y_train)
test_score = model.score(X_test_scaled, y_test)
print(f"Train R^2: {train_score:.3f}")
print(f"Test R^2:  {test_score:.3f}")


Train R^2: 0.486
Test R^2:  0.436


In [50]:
# Predict on the test set
y_pred = model.predict(X_test_scaled)

# Print the first 5 predictions vs. the actual values
print("\nSample Predictions (first 5):")
for i in range(5):
    print(f"Sample {i+1}:")
    print(f"  Predicted: {y_pred[i]}")
    print(f"  Actual:    {y_test[i]}")
    print()



Sample Predictions (first 5):
Sample 1:
  Predicted: [1.02399899e+03 4.83821844e-01 1.86707159e+01 6.64389960e+01
 7.13777160e+00 1.28991015e+02 1.41146462e+02 1.03326083e+01
 4.60617758e+00]
  Actual:    [1.13987917e+03 8.66666667e-01 2.39212500e+01 5.61020833e+01
 9.25916667e+00 1.29892083e+02 1.43832500e+02 1.20337500e+01
 5.00000000e+00]

Sample 2:
  Predicted: [1.10359781e+03 5.09769841e-01 2.09187248e+01 6.79723639e+01
 7.57644450e+00 1.31022762e+02 1.43461418e+02 1.03389499e+01
 4.62440634e+00]
  Actual:    [1.11790500e+03 3.64583333e-01 2.51258333e+01 6.05579167e+01
 7.86166667e+00 1.11236250e+02 1.22392500e+02 7.05583333e+00
 5.00000000e+00]

Sample 3:
  Predicted: [1.15736050e+03 4.85717088e-01 2.23969430e+01 6.87141961e+01
 7.76474054e+00 1.28602054e+02 1.41618839e+02 1.03910325e+01
 4.58361484e+00]
  Actual:    [1.11790500e+03 3.64583333e-01 2.51258333e+01 6.05579167e+01
 7.86166667e+00 1.11236250e+02 1.22392500e+02 7.05583333e+00
 5.00000000e+00]

Sample 4:
  Predicted: [

In [51]:
import json

# Extract the weights (coefs and intercepts) and convert them to regular lists
model_weights = {
    "coefs": [coef.tolist() for coef in model.coefs_],
    "intercepts": [inter.tolist() for inter in model.intercepts_]
}

# Write the weights to a JSON file
with open("model_weights.json", "w") as f:
    json.dump(model_weights, f)

print("Model weights saved to model_weights.json")


Model weights saved to model_weights.json


In [52]:
import json
import numpy as np

# Load the weights from the JSON file
with open("model_weights.json", "r") as f:
    model_weights = json.load(f)

# Convert lists back to NumPy arrays
coefs = [np.array(coef) for coef in model_weights["coefs"]]
intercepts = [np.array(inter) for inter in model_weights["intercepts"]]

print("Loaded model weights successfully!")

# Example: Print the first layer's weights and biases
print("\nFirst layer weights shape:", coefs[0].shape)
print("First layer intercepts shape:", intercepts[0].shape)


Loaded model weights successfully!

First layer weights shape: (63, 16)
First layer intercepts shape: (16,)


In [53]:
coefs[0]

array([[-2.61193238e-02, -9.71270652e-01, -3.46079139e-01, ...,
         4.16546323e-01, -6.21646055e-01, -2.07714404e+00],
       [-2.89019437e+00,  9.57951014e-03, -1.73406947e+00, ...,
         3.64794102e-01, -1.47359907e-01, -1.37178816e+00],
       [-9.12443606e-02, -1.71172411e+00, -3.14861750e-03, ...,
         9.24396455e-01,  1.27385300e+00,  9.08229678e-02],
       ...,
       [-3.70711751e+00,  4.77366754e-01,  1.69616381e+00, ...,
         2.24678768e+00, -4.01604740e+00,  4.07078923e+00],
       [ 2.63898944e-01,  1.40380704e+00,  1.33720212e+00, ...,
        -2.08476986e-02, -2.40721137e-01,  2.91362572e+00],
       [-1.08875783e+00, -2.14894087e+00,  3.14088233e+00, ...,
         2.65769396e+00, -1.53403473e+00,  3.45718885e+00]])